In [1]:
import warnings
from pandas.errors import PerformanceWarning

warnings.simplefilter(action="ignore", category=PerformanceWarning)
warnings.simplefilter(action="ignore", category=RuntimeWarning)

# 3. Агрегирование оценок

In [2]:
import pandas as pd
import numpy as np
from collections import defaultdict
from tqdm import tqdm

In [3]:
df = pd.read_csv(
    "paragrapghs_with_topics.csv.zip",
    index_col=0,
    engine="pyarrow",
    usecols=[
        "Unnamed: 0",
        "company",
        "year",
        "Экология в целом",
        "Климат",
        "Энергия",
        "Воздух",
        "Вода",
        "Отходы и циклическая экономика",
        "Биоразнообразие",
        "Рекультивация земель",
        "Экологичность продукта",
        "!Персонал в целом",
        "Обучение и развитие",
        "Сотрудники. Вовлеченность и мот",
        "Оплата труда",
        "Сотрудники. Здоровье и благопол",
        "Сотрудники. Наем и увольнение",
        "Сотрудники. Корпоративная культ",
        "Сотрудники. Безопасность и охра",
        "Сотрудники. Профсоюз и Коллекти",
        "Потребители. Доступность",
        "Потребители. Сервис и коммуника",
        "Потребители. Персональные данны",
        "Потребители. Здоровье и благопо",
        "Потребители. Маркетинг, продажи",
        "Потребители. Удовлетворенность",
        "Потребители. Качество и безопас",
        "Потребители. Ценовая политика",
        "Поставщики в целом",
        "Малый и локальный бизнес",
        "Поставщики. Работники",
        "Поставщики. Экология",
        "Закупки и антикоррупция",
        "Заинтересованные стороны",
        "Коренные народы и местные сообщ",
        "Сотрудники. Волонтерство",
        "Социальные инвестиции и благотв",
        "Отчетность и прозрачность",
        "Отношения с инвесторами",
        "!Инновации",
        "Права человека",
        "Лидерство",
        "Риски",
        "Этика",
        "Корпоративное управление",
        "Устойчивое развитие",
        "!Кибербезопасность",
        "max_topic_cos",
        "max_score_cos",
    ],
)
df.head()

,company,year,Экология в целом,Климат,Энергия,Воздух,Вода,Отходы и циклическая экономика,Биоразнообразие,Рекультивация земель,...,!Инновации,Права человека,Лидерство,Риски,Этика,Корпоративное управление,Устойчивое развитие,!Кибербезопасность,max_topic_cos,max_score_cos
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
0.0,2019/ПАО «Ростелеком»,2019,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.179605,0.0,Обучение и развитие,0.214768
1.0,2019/ПАО «Ростелеком»,2019,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.179605,0.0,Обучение и развитие,0.214768
2.0,2019/ПАО «Ростелеком»,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN
3.0,2019/ПАО «Ростелеком»,2019,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.179605,0.0,Обучение и развитие,0.214768
4.0,2019/ПАО «Ростелеком»,2019,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.295491,0.0,0.0,0.0,0.000000,0.0,Лидерство,0.295491


In [5]:
not_nan = df[df["max_topic_cos"].notna()]
# число абзацев соотвествующих теме
reports = not_nan.groupby(["year", "company", "max_topic_cos"])["max_score_cos"].count()
reports.head()

year        company            max_topic_cos           
2002, 2001  2002/«БАТ Россия»                              0
                               Воздух                      1
                               Закупки и антикоррупция     2
                               Климат                      1
                               Корпоративное управление    2
Name: max_score_cos, dtype: int64

# TF-IDF

## TF

In [6]:
topics_per_report = not_nan.groupby(["year", "company"])["max_score_cos"].count()
topics_per_report.head()

year        company                                                       
2002, 2001  2002/«БАТ Россия»                                                 203
2003        2003/АО «Альфа-банк»                                               44
            2003/Концерн «Шелл»                                               131
            2003/ООО «Газпром трансгаз Ухта» (ООО «Севергазпром»)             152
            2003/ООО «Неманский целлюлозно-бумажный комбинат» (ЗАО «СЗЛК»)      6
Name: max_score_cos, dtype: int64

In [7]:
tf = defaultdict(dict)
for (year, company, topic), row in tqdm(reports.items(), total=reports.shape[0]):
    tf[(year, company)][topic] = row / topics_per_report[year][company]
tf = pd.DataFrame(tf).T

  0%|          | 0/40479 [00:00<?, ?it/s]

100%|██████████| 40479/40479 [00:04<00:00, 8579.40it/s]


## IDF

In [8]:
tf.head()

Воздух  \
2002, 2001 2002/«БАТ Россия»                                   0.0  0.004926   
2003       2003/АО «Альфа-банк»                                0.0       NaN   
           2003/Концерн «Шелл»                                 0.0  0.015267   
           2003/ООО «Газпром трансгаз Ухта» (ООО «Севергаз...  0.0  0.177632   
           2003/ООО «Неманский целлюлозно-бумажный комбина...  0.0       NaN   

                                                               Закупки и антикоррупция  \
2002, 2001 2002/«БАТ Россия»                                                  0.009852   
2003       2003/АО «Альфа-банк»                                                    NaN   
           2003/Концерн «Шелл»                                                0.007634   
           2003/ООО «Газпром трансгаз Ухта» (ООО «Севергаз...                 0.006579   
           2003/ООО «Неманский целлюлозно-бумажный комбина...                      NaN   

                                                                 Климат  \
2002, 2001 2002/«БАТ Россия»                                   0.004926   
2003       2003/АО «Альфа-банк»                                     NaN   
           2003/Концерн «Шелл»                                 0.030534   
           2003/ООО «Газпром трансгаз Ухта» (ООО «Севергаз...  0.013158   
           2003/ООО «Неманский целлюлозно-бумажный комбина...       NaN   

                                                               Корпоративное управление  \
2002, 2001 2002/«БАТ Россия»                                                   0.009852   
2003       2003/АО «Альфа-банк»                                                     NaN   
           2003/Концерн «Шелл»                                                      NaN   
           2003/ООО «Газпром трансгаз Ухта» (ООО «Севергаз...                  0.006579   
           2003/ООО «Неманский целлюлозно-бумажный комбина...                       NaN   

                                                               Лидерство  \
2002, 2001 2002/«БАТ Россия»                                    0.044335   
2003       2003/АО «Альфа-банк»                                 0.181818   
           2003/Концерн «Шелл»                                  0.022901   
           2003/ООО «Газпром трансгаз Ухта» (ООО «Севергаз...   0.032895   
           2003/ООО «Неманский целлюлозно-бумажный комбина...        NaN   

                                                               Обучение и развитие  \
2002, 2001 2002/«БАТ Россия»                                              0.019704   
2003       2003/АО «Альфа-банк»                                           0.045455   
           2003/Концерн «Шелл»                                            0.015267   
           2003/ООО «Газпром трансгаз Ухта» (ООО «Севергаз...             0.013158   
           2003/ООО «Неманский целлюлозно-бумажный комбина...                  NaN   

                                                               Оплата труда  \
2002, 2001 2002/«БАТ Россия»                                       0.019704   
2003       2003/АО «Альфа-банк»                                         NaN   
           2003/Концерн «Шелл»                                          NaN   
           2003/ООО «Газпром трансгаз Ухта» (ООО «Севергаз...           NaN   
           2003/ООО «Неманский целлюлозно-бумажный комбина...           NaN   

                                                               Отходы и циклическая экономика  \
2002, 2001 2002/«БАТ Россия»                                                         0.162562   
2003       2003/АО «Альфа-банк»                                                           NaN   
           2003/Концерн «Шелл»                                                       0.030534   
           2003/ООО «Газпром трансгаз Ухта» (ООО «Севергаз...                        0.164474   
           2003/ООО «Неманский целлюлозно-бумажный комбина...                        1.000000   

                  

In [9]:
idf = {}
for topic in tqdm(df["max_topic_cos"].unique()):
    if pd.isna(topic):
        continue
    idf[topic] = np.log(df["company"].nunique() / tf[topic].notna().sum())
idf = pd.Series(idf)

100%|██████████| 41/41 [00:13<00:00,  3.12it/s]


In [10]:
tf_idf = defaultdict(dict)
for idx, row in tf.iterrows():
    for topic in row.index:
        tf_idf[idx][topic] = row[topic] * idf[topic]

In [11]:
tf_idf = pd.DataFrame(tf_idf).T
tf_idf = tf_idf.fillna(0)
tf_idf.head()

Воздух  \
2002, 2001 2002/«БАТ Россия»                                  -0.0  0.001023   
2003       2003/АО «Альфа-банк»                               -0.0  0.000000   
           2003/Концерн «Шелл»                                -0.0  0.003171   
           2003/ООО «Газпром трансгаз Ухта» (ООО «Севергаз... -0.0  0.036895   
           2003/ООО «Неманский целлюлозно-бумажный комбина... -0.0  0.000000   

                                                               Закупки и антикоррупция  \
2002, 2001 2002/«БАТ Россия»                                                  0.001621   
2003       2003/АО «Альфа-банк»                                               0.000000   
           2003/Концерн «Шелл»                                                0.001256   
           2003/ООО «Газпром трансгаз Ухта» (ООО «Севергаз...                 0.001083   
           2003/ООО «Неманский целлюлозно-бумажный комбина...                 0.000000   

                                                                 Климат  \
2002, 2001 2002/«БАТ Россия»                                   0.000921   
2003       2003/АО «Альфа-банк»                                0.000000   
           2003/Концерн «Шелл»                                 0.005708   
           2003/ООО «Газпром трансгаз Ухта» (ООО «Севергаз...  0.002460   
           2003/ООО «Неманский целлюлозно-бумажный комбина...  0.000000   

                                                               Корпоративное управление  \
2002, 2001 2002/«БАТ Россия»                                                   0.001281   
2003       2003/АО «Альфа-банк»                                                0.000000   
           2003/Концерн «Шелл»                                                 0.000000   
           2003/ООО «Газпром трансгаз Ухта» (ООО «Севергаз...                  0.000855   
           2003/ООО «Неманский целлюлозно-бумажный комбина...                  0.000000   

                                                               Лидерство  \
2002, 2001 2002/«БАТ Россия»                                    0.001421   
2003       2003/АО «Альфа-банк»                                 0.005827   
           2003/Концерн «Шелл»                                  0.000734   
           2003/ООО «Газпром трансгаз Ухта» (ООО «Севергаз...   0.001054   
           2003/ООО «Неманский целлюлозно-бумажный комбина...   0.000000   

                                                               Обучение и развитие  \
2002, 2001 2002/«БАТ Россия»                                              0.000666   
2003       2003/АО «Альфа-банк»                                           0.001537   
           2003/Концерн «Шелл»                                            0.000516   
           2003/ООО «Газпром трансгаз Ухта» (ООО «Севергаз...             0.000445   
           2003/ООО «Неманский целлюлозно-бумажный комбина...             0.000000   

                                                               Оплата труда  \
2002, 2001 2002/«БАТ Россия»                                       0.003522   
2003       2003/АО «Альфа-банк»                                    0.000000   
           2003/Концерн «Шелл»                                     0.000000   
           2003/ООО «Газпром трансгаз Ухта» (ООО «Севергаз...      0.000000   
           2003/ООО «Неманский целлюлозно-бумажный комбина...      0.000000   

                                                               Отходы и циклическая экономика  \
2002, 2001 2002/«БАТ Россия»                                                         0.010003   
2003       2003/АО «Альфа-банк»                                                      0.000000   
           2003/Концерн «Шелл»                                                       0.001879   
           2003/ООО «Газпром трансгаз Ухта» (ООО «Севергаз...                        0.010120   
           2003/ООО «Неманский целлюлозно-бумажный комбина...                        0.061531   

                  

In [12]:
tf_idf.to_csv("tf_idf.csv")

In [13]:
topic_words = pd.read_csv("../topic_words.csv", index_col=0)
topic_words.head()

,topic,weight,type,word
meta,,,,
env,Экология в целом,5.0,0.0,экологичный
env,Экология в целом,5.0,0.0,природоохранный
env,Экология в целом,1.0,0.0,экология
env,Экология в целом,5.0,0.0,воздействие
env,Экология в целом,5.0,0.0,окружающая среда


In [14]:
gr = topic_words.groupby("meta")["topic"].apply(set)

# TOPSIS

In [15]:
from pymcdm.methods import TOPSIS

/home/samoed/.pyenv/versions/3.10.4/lib/python3.10/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [17]:
gr = (
    df.drop(
        columns=[
            "max_topic_cos",
            "max_score_cos",
        ]
    )
    .groupby(["year", "company"])
    .mean()
    .dropna(how="all")
)
gr.head()

Экология в целом  \
year       company                                                                
2002, 2001 2002/«БАТ Россия»                                           0.000000   
2003       2003/АО «Альфа-банк»                                        0.000000   
           2003/Концерн «Шелл»                                         0.000000   
           2003/ООО «Газпром трансгаз Ухта» (ООО «Севергаз...          0.025313   
           2003/ООО «Неманский целлюлозно-бумажный комбина...          0.000000   

                                                                 Климат  \
year       company                                                        
2002, 2001 2002/«БАТ Россия»                                   0.001100   
2003       2003/АО «Альфа-банк»                                0.000000   
           2003/Концерн «Шелл»                                 0.010146   
           2003/ООО «Газпром трансгаз Ухта» (ООО «Севергаз...  0.020184   
           2003/ООО «Неманский целлюлозно-бумажный комбина...  0.000000   

                                                                Энергия  \
year       company                                                        
2002, 2001 2002/«БАТ Россия»                                   0.000000   
2003       2003/АО «Альфа-банк»                                0.000000   
           2003/Концерн «Шелл»                                 0.025629   
           2003/ООО «Газпром трансгаз Ухта» (ООО «Севергаз...  0.001722   
           2003/ООО «Неманский целлюлозно-бумажный комбина...  0.000000   

                                                                 Воздух  \
year       company                                                        
2002, 2001 2002/«БАТ Россия»                                   0.001153   
2003       2003/АО «Альфа-банк»                                0.000000   
           2003/Концерн «Шелл»                                 0.006271   
           2003/ООО «Газпром трансгаз Ухта» (ООО «Севергаз...  0.059533   
           2003/ООО «Неманский целлюлозно-бумажный комбина...  0.000000   

                                                                   Вода  \
year       company                                                        
2002, 2001 2002/«БАТ Россия»                                   0.003818   
2003       2003/АО «Альфа-банк»                                0.000000   
           2003/Концерн «Шелл»                                 0.001479   
           2003/ООО «Газпром трансгаз Ухта» (ООО «Севергаз...  0.038040   
           2003/ООО «Неманский целлюлозно-бумажный комбина...  0.000000   

                                                               Отходы и циклическая экономика  \
year       company                                                                              
2002, 2001 2002/«БАТ Россия»                                                         0.030931   
2003       2003/АО «Альфа-банк»                                                      0.000000   
           2003/Концерн «Шелл»                                                       0.007189   
           2003/ООО «Газпром трансгаз Ухта» (ООО «Севергаз...                        0.045638   
           2003/ООО «Неманский целлюлозно-бумажный комбина...                        0.181323   

                                                               Биоразнообразие  \
year       company                                                               
2002, 2001 2002/«БАТ Россия»                                          0.000000   
2003       2003/АО «Альфа-банк»                                       0.000000   
           2003/Концерн «Шелл»                                        0.003433   
           2003/ООО «Газпром трансгаз Ухта» (ООО «Севергаз...         0.024522   
           2003/ООО «Неманский целлюлозно-бумажный комбина...         0.000000   

                                                               Рекультивация земель  \
year       company                 

In [18]:
topics = topic_words.groupby("meta")["topic"].apply(set)

In [19]:
gr.reset_index(inplace=True)
gr.head()

,year,company,Экология в целом,Климат,Энергия,Воздух,Вода,Отходы и циклическая экономика,Биоразнообразие,Рекультивация земель,...,Отчетность и прозрачность,Отношения с инвесторами,!Инновации,Права человека,Лидерство,Риски,Этика,Корпоративное управление,Устойчивое развитие,!Кибербезопасность
0,"2002, 2001",2002/«БАТ Россия»,0.000000,0.001100,0.000000,0.001153,0.003818,0.030931,0.000000,0.002968,...,0.071654,0.000000,0.0,0.0,0.018783,0.004526,0.0,0.004193,0.000868,0.0
1,2003,2003/АО «Альфа-банк»,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.103845,0.010507,0.0,0.0,0.043749,0.004539,0.0,0.000000,0.000000,0.0
2,2003,2003/Концерн «Шелл»,0.000000,0.010146,0.025629,0.006271,0.001479,0.007189,0.003433,0.000000,...,0.112553,0.000000,0.0,0.0,0.008659,0.017162,0.0,0.000000,0.030949,0.0
3,2003,2003/ООО «Газпром трансгаз Ухта» (ООО «Северга...,0.025313,0.020184,0.001722,0.059533,0.038040,0.045638,0.024522,0.017700,...,0.007528,0.000000,0.0,0.0,0.008142,0.007105,0.0,0.002800,0.001159,0.0
4,2003,2003/ООО «Неманский целлюлозно-бумажный комбин...,0.000000,0.000000,0.000000,0.000000,0.000000,0.181323,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0


In [20]:
def second_workflow(data: pd.DataFrame):
    matrix = data.to_numpy()
    weights = np.array([1 / matrix.shape[0]] * matrix.shape[1])
    types = np.ones(matrix.shape[1], dtype=int)
    body = TOPSIS()
    predictions = [round(preference, 2) for preference in body(matrix, weights, types)]
    return predictions

In [21]:
bad = ["Кибербезопасность", "Инновации", "Персонал в целом"]

for year in gr["year"].unique():
    year_data = gr[gr["year"] == year]
    for key, cur_topics in topics.items():
        tmp_topics = []
        for t in cur_topics:
            if t not in bad:
                tmp_topics.append(t)
            else:
                tmp_topics.append("!" + t)
        cur_topics = tmp_topics
        res = second_workflow(year_data[cur_topics])
        res = pd.Series(res).set_axis(year_data.index)
        gr.loc[gr["year"] == year, key] = res

In [22]:
bad = ["Кибербезопасность", "Инновации", "Персонал в целом"]

for year in gr["year"].unique():
    year_data = gr[gr["year"] == year]
    tmp_topics = []
    for t in [v for _, val in topics.items() for v in val]:
        if t not in bad:
            tmp_topics.append(t)
        else:
            tmp_topics.append("!" + t)
    cur_topics = tmp_topics
    res = second_workflow(year_data[cur_topics])
    res = pd.Series(res).set_axis(year_data.index)
    gr.loc[gr["year"] == year, "all_letters"] = res
    res = pd.Series(second_workflow(year_data[["env", "gov", "social"]])).set_axis(year_data.index)
    gr.loc[gr["year"] == year, "on_letters"] = res

In [23]:
gr["avg"] = (gr["env"] + gr["gov"] + gr["social"]) / 3

In [24]:
gr = gr.sort_values(["year", "avg"], ascending=[True, False])

In [25]:
gr["company"] = gr["company"].str.split("/").str[1]

In [26]:
gr.to_csv("results.csv")

# Plot

In [27]:
import plotly.express as px
import plotly.graph_objects as go

In [28]:
df = pd.read_csv("../results.csv", index_col=0)

In [29]:
new_df = []

for idx, row in tqdm(df.iterrows(), total=df.shape[0]):
    years = row["year"]
    # row["idx"] = idx

    if years.find(",") != -1:
        for year in years.split(","):
            new_row = row.copy()
            new_row["year"] = int(year)
            new_df.append(new_row)
    else:
        row["year"] = float(row["year"])
        new_df.append(row)

100%|██████████| 1244/1244 [00:00<00:00, 7326.07it/s]


In [30]:
df = pd.DataFrame(new_df).reset_index(drop=True)
df["year"] = df["year"].astype(int)
df.head()

,year,company,Unnamed: 0topics,Экология в целом,Климат,Энергия,Воздух,Вода,Отходы и циклическая экономика,Биоразнообразие,...,Этика,Корпоративное управление,Устойчивое развитие,!Кибербезопасность,env,gov,social,all_letters,avg,on_letters
0,2004,АО «Группа «Илим»,2.670868e+06,0.000000,0.005321,0.012626,0.003872,0.030120,0.020988,0.015211,...,0.000000,0.003489,0.006082,0.0,0.50,0.67,0.56,0.56,0.576667,1.00
1,2004,ОАО «Рязанская ГРЭС»,2.670694e+06,0.000000,0.000000,0.000000,0.031644,0.028729,0.023511,0.015468,...,0.000000,0.000000,0.000000,0.0,0.50,0.33,0.44,0.44,0.423333,0.00
2,2006,ОАО «ТГК-6»,3.071506e+06,0.012101,0.012886,0.052351,0.018976,0.026239,0.012565,0.003150,...,0.000000,0.010256,0.006888,0.0,0.47,0.48,0.39,0.43,0.446667,0.85
3,2006,ОАО «Волжская ТГК»,3.074424e+06,0.003978,0.012979,0.059898,0.026394,0.026344,0.022817,0.009506,...,0.000000,0.009659,0.007815,0.0,0.43,0.49,0.38,0.41,0.433333,0.81
4,2006,BP p.l.c.,3.075012e+06,0.012450,0.045558,0.028783,0.009554,0.006310,0.008558,0.005282,...,0.002169,0.002895,0.021662,0.0,0.38,0.53,0.37,0.40,0.426667,0.77


In [31]:
df = df.groupby(["year", "company"])[["env", "gov", "social", "all_letters", "avg", "on_letters"]].mean()

In [32]:
unstack_df = df.unstack(level=0)
unstack_df.head()

env                                                    ...  \
year         2000 2001 2002 2003  2004  2005  2006  2007 2008  2009  ...   
company                                                              ...   
Abbott        NaN  NaN  NaN  NaN   NaN   NaN   NaN   NaN  NaN   NaN  ...   
Alcoa, Inc.   NaN  NaN  NaN  NaN  0.31  0.35  0.40   NaN  NaN   NaN  ...   
BELUGA GROUP  NaN  NaN  NaN  NaN   NaN   NaN   NaN   NaN  NaN   NaN  ...   
BIOCAD        NaN  NaN  NaN  NaN   NaN   NaN   NaN   NaN  NaN   NaN  ...   
BP p.l.c.     NaN  NaN  NaN  NaN  0.43  0.28  0.38  0.34  0.3  0.29  ...   

             on_letters                                                   
year               2013 2014 2015 2016 2017  2018  2019  2020  2021 2022  
company                                                                   
Abbott              0.0  NaN  NaN  NaN  NaN   NaN   NaN   NaN   NaN  NaN  
Alcoa, Inc.         NaN  NaN  NaN  NaN  NaN   NaN   NaN   NaN   NaN  NaN  
BELUGA GROUP        NaN  NaN  NaN  NaN  NaN   NaN   NaN  0.59   NaN  NaN  
BIOCAD              NaN  NaN  NaN  NaN  NaN  0.46  0.46  0.41  0.41  NaN  
BP p.l.c.           NaN  NaN  NaN  NaN  NaN   NaN   NaN   NaN   NaN  NaN  

[5 rows x 138 columns]

In [33]:
unstack_df.to_csv("results_all.csv")

In [34]:
for col in ["all_letters", "avg", "on_letters"]:
    df.groupby(["year", "company"])[col].mean().unstack(level=0).to_csv(f"{col}.csv")

In [35]:
fig = go.Figure()

# Add Traces
cols = ["all_letters", "avg", "on_letters"]
size = defaultdict(int)
for col in cols:
    for d in px.line(unstack_df[col].T).data:
        fig.add_trace(d)
        size[col] += 1

visible = defaultdict(list)
for col in cols:
    visible[col] = [cols[0] == col] * size[col] + [cols[1] == col] * size[col] + [cols[2] == col] * size[col]

fig.update_layout(
    updatemenus=[
        dict(
            active=0,
            buttons=list(
                [
                    dict(
                        label=col,
                        method="update",
                        args=[
                            {"visible": visible[col]},
                            {
                                "title": col,
                            },
                        ],
                    )
                    for col in cols
                ]
            ),
        )
    ]
)

# Set title
fig.update_layout(title=cols[0], xaxis_title="Года", yaxis_title="Оценка")

fig.show()